# Detect Cars Tutorial
This notebook shows how to detect cars in a folder of images using Moondream.
You'll see how to annotate a single image and then process an entire folder,
saving the overlayed images and a JSON summary.

In [ ]:
import os
import json
from PIL import Image, ImageDraw
from IPython.display import display
import moondream as md

# Initialize the model to run locally
model = md.vl(endpoint="http://localhost:2020/v1")
# For Moondream Cloud use:
# model = md.vl(api_key="<your-api-key>")

INPUT_FOLDER = os.path.join(os.path.dirname(__file__), '..', '..', 'images')
OUTPUT_FOLDER = os.path.join(INPUT_FOLDER, 'detected')
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

image_files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(('png','jpg','jpeg'))]

## Detect a single image
Let's run detection on the first image and display the result with bounding boxes.

In [ ]:
first_path = os.path.join(INPUT_FOLDER, image_files[0])
first_image = Image.open(first_path)
detection = model.detect(first_image, 'car')['objects']
overlay = first_image.copy()
draw = ImageDraw.Draw(overlay)
for box in detection:
    draw.rectangle([box['x_min'], box['y_min'], box['x_max'], box['y_max']], outline='red', width=3)
display(overlay)
detection

## Process the entire folder
Now let's loop through all the images, saving overlayed versions and recording the results in a JSON file.

In [ ]:
results = []
for fname in image_files:
    img_path = os.path.join(INPUT_FOLDER, fname)
    img = Image.open(img_path)
    boxes = model.detect(img, 'car')['objects']
    ov = img.copy()
    d = ImageDraw.Draw(ov)
    for b in boxes:
        d.rectangle([b['x_min'], b['y_min'], b['x_max'], b['y_max']], outline='red', width=3)
    out_path = os.path.join(OUTPUT_FOLDER, fname)
    ov.save(out_path)
    results.append({'original': img_path, 'overlay': out_path, 'boxes': boxes})

json_path = os.path.join(OUTPUT_FOLDER, 'results.json')
with open(json_path, 'w') as f:
    json.dump(results, f, indent=2)

f'Saved {len(results)} results to {json_path}'